### OpenAI CLIP으로 이미지 분류하기

### 학습 내용
 * 실습을 위한 사전 준비 및 확인
 * OpenAI CLIP 이미지 분류하기 전체 실행 코드
 * 역방향 안정적 확산 : 이미지를 텍스트로 변환하기

### 실습을 위한 사전 준비 및 확인
  * pip install openai 설치
  * 코랩 노트북 런타임 유형 GPU로 변경
  * 이미지 다운로드
     * wget https://upload.wikimedia.org/wikipedia/commons/d/d0/STS086-371-015_-_STS-086_-_Various_views_of_STS-86_and_Mir_24_crewmembers_on_the_Mir_space_station_-_DPLA_-_92233a2e397bd089d70a7fcf922b34a4.jpg

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00


In [2]:
!wget -O image01.jpg https://upload.wikimedia.org/wikipedia/commons/d/d0/STS086-371-015_-_STS-086_-_Various_views_of_STS-86_and_Mir_24_crewmembers_on_the_Mir_space_station_-_DPLA_-_92233a2e397bd089d70a7fcf922b34a4.jpg

--2023-08-31 07:00:40--  https://upload.wikimedia.org/wikipedia/commons/d/d0/STS086-371-015_-_STS-086_-_Various_views_of_STS-86_and_Mir_24_crewmembers_on_the_Mir_space_station_-_DPLA_-_92233a2e397bd089d70a7fcf922b34a4.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 185.15.59.240, 2a02:ec80:300:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|185.15.59.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1843795 (1.8M) [image/jpeg]
Saving to: ‘image01.jpg’

image01.jpg         100%[===================>]   1.76M  6.58MB/s    in 0.3s    

2023-08-31 07:00:40 (6.58 MB/s) - ‘image01.jpg’ saved [1843795/1843795]



#### CUDA인지 확인하기

In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


#### clip.load() 함수를 이용한 모델 로드

In [5]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-j3lszw1a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-j3lszw1a
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=644d0bc8369fb6814301a1a4ce29950411557ece02518c302ed1845fa1232543
  Stored in directory: /tmp/pip-ephem-wheel-cache-ax0zcjor/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [6]:
import clip
model, preprocess = clip.load('ViT-B/32', device=device)
print(model, '\n preprocess :', preprocess)

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 57.1MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

#### CLIP 모델을 사용한 인코딩

In [7]:
import PIL

In [8]:
# Load an image
# 이미지 불러오기
image = PIL.Image.open("image01.jpg")

# 이미지 전처리
image_input = preprocess(image).unsqueeze(0).to(device)

# CLIP 모델을 사용하여 이미지 인코드
with torch.no_grad():
    image_features = model.encode_image(image_input)


#### 텍스트 프롬프트 리스트 정의

In [9]:
prompts = [
    "A large galaxy in the center of a cluster of galaxies located in the constellation Bostes",
    "MTA Long Island Bus has just left the Hempstead Bus Terminal on the N6",
    "STS-86 mission specialists Vladimir Titov and Jean-Loup Chretien pose for photos in the Base Block",
    "A view of the International Space Station (ISS) from the Soyuz TMA-19 spacecraft, as it approaches the station for docking",
    "A domesticated tiger in a cage with a tiger trainer in the background",
    "A mechanical engineer working on a car engine",
]

# CLIP 모델을 사용한 텍스트 프롬프트 인코드
with torch.no_grad():
    text_features = model.encode_text(clip.tokenize(prompts).to(device))

#### 이미지와 각 프롬프트 사이의 유사도 계산

In [10]:
# 이미지와 각각의 프롬프트 사이의 유사도 계산하기
similarity_scores = (100.0 * image_features @ text_features.T).softmax(dim=-1)
print(similarity_scores)


tensor([[0., 0., 1., 0., 0., 0.]], device='cuda:0', dtype=torch.float16)


In [11]:
# 유사도 점수가 가장 높은 프롬프트를 출력
most_similar_prompt_index = similarity_scores.argmax().item()
most_similar_prompt = prompts[most_similar_prompt_index]
print("The image is most similar to the prompt: {} ".format(most_similar_prompt))


The image is most similar to the prompt: STS-86 mission specialists Vladimir Titov and Jean-Loup Chretien pose for photos in the Base Block 


### OpenAI CLIP 이미지 분류하기 전체 실행 코드

In [12]:
# 필수 라이브러리 불러오기
import torch
import clip
import PIL

# CLIP 모델 불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)

# 이미지 불러오기
image = PIL.Image.open("image01.jpg")

# 이미지 전처리
image_input = preprocess(image).unsqueeze(0).to(device)

# CLIP 모델을 사용한 이미지 인코드
with torch.no_grad():
    image_features = model.encode_image(image_input)

# 텍스트 프롬프트 리스트 정의하기
prompts = [
    "A large galaxy in the center of a cluster of galaxies located in the constellation Bostes",
    "MTA Long Island Bus has just left the Hempstead Bus Terminal on the N6",
    "STS-86 mission specialists Vladimir Titov and Jean-Loup Chretien pose for photos in the Base Block",
    "A view of the International Space Station (ISS) from the Soyuz TMA-19 spacecraft, as it approaches the station for docking",
    "A domesticated tiger in a cage with a tiger trainer in the background",
    "A mechanical engineer working on a car engine",
]

# CLIP 모델을 사용한 텍스트 프롬프트 인코드
with torch.no_grad():
    text_features = model.encode_text(clip.tokenize(prompts).to(device))

# 이미지와 각각의 텍스트 프롬프트 사이의 유사도 계산하기
similarity_scores = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# 가장 높은 유사도 점수와 프롬프트 출력하기
most_similar_prompt_index = similarity_scores.argmax().item()
most_similar_prompt = prompts[most_similar_prompt_index]
print("The image is most similar to the prompt: {}".format(most_similar_prompt))


The image is most similar to the prompt: STS-86 mission specialists Vladimir Titov and Jean-Loup Chretien pose for photos in the Base Block


### 역방향 안정적 확산 : 이미지를 텍스트로 변환하기

In [13]:
!pip install clip-interrogator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 787.8/787.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 98.1 MB/s eta 0:00:00


#### 이미지를 텍스트로 변경하는 작업 수행

In [14]:
from PIL import Image
from clip_interrogator import Config, Interrogator

# 이미지 경로 지정
image_path = 'image01.jpg'
image = Image.open(image_path).convert('RGB')

ci = Interrogator(Config(clip_model_name = "ViT-L-14/openai"))

print(ci.interrogate(image))

Loading caption model blip-large...


Loading CLIP model ViT-L-14/openai...


100%|███████████████████████████████████████| 933M/933M [00:17<00:00, 54.3MiB/s]
ViT-L-14_openai_artists.safetensors: 100%|██████████| 16.2M/16.2M [00:00<00:00, 268MB/s]
ViT-L-14_openai_flavors.safetensors: 100%|██████████| 155M/155M [00:00<00:00, 315MB/s]
ViT-L-14_openai_mediums.safetensors: 100%|██████████| 146k/146k [00:00<00:00, 11.7MB/s]
ViT-L-14_openai_movements.safetensors: 100%|██████████| 307k/307k [00:00<00:00, 18.8MB/s]
ViT-L-14_openai_trendings.safetensors: 100%|██████████| 111k/111k [00:00<00:00, 12.7MB/s]
ViT-L-14_openai_negative.safetensors: 100%|██████████| 63.2k/63.2k [00:00<00:00, 56.5MB/s]


Loaded CLIP model and data in 29.97 seconds.


100%|██████████| 50/50 [00:00<00:00, 185.57it/s]

three men in overalls standing in a space station with a monitor, macron with afro hair, without beard and mustache, inspired by Mikhail Evstafiev, in the 1986 vert contest, taking control while smiling, moustache, the backroom, technical vest, from 2001, camper, 2009, 2005
